# 1. Dirichlet fitting directly on MC2010

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

#Graphical parameters
mrkSz=5
pltSlcFactor=100

import fitKelvinChain_v2025 as fkc
import creepMC2010 as MC2010
import numpy as np
import matplotlib.pyplot as plt

cm = 1/2.54
fig, ax = plt.subplots(1, 1, figsize=(8*cm, 11*cm), constrained_layout=True)
fontSize = 8
plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = MC2010.creepMC2010(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=1
numCreepPoints=10 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=1 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=10
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints)) 
#Initialize compliance vector
complianceSeries=[]
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))

for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Eci_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Eci for t in timeSpan]
    initialStrain=[1/costConcrete.Eci_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=[value for value in timeSpan]
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    np.random.seed(12)
    #[J[enum].append(element+np.random.normal(np.random.normal(0,0.15*element,1)[0],0.15*element, 1)[0]) for element in yaxis]
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])

    #Visualize data being stored, if you wish
    c = next(color)
    ax.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="MC10-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)


#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel = fkc.kelvinChainModel(ageingComplianceSeries)

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax)

ax.set_xlabel("Days since loading (days)")
ax.set_ylabel("Compliance ($MPa^{-1}$)")
ax.set_xscale('log')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, columnspacing=0.2, ncol=2, fontsize=fontSize)
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.subplots_adjust(wspace=0.0, hspace=0, right=.95)
fig.tight_layout()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\renan\AppData\Local\Temp\ipykernel_1476\2588706601.py:88: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  fig.subplots_adjust(wspace=0.0, hspace=0, right=.95)
C:\Users\renan\AppData\Local\Temp\ipykernel_1476\2588706601.py:89: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


# 2. Dirichlet fitting through a modified DPL smoothing a EC2 compliance with artificial noise

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import fitKelvinChain_v2025 as fkc
import creepEC2 as cEC2
import numpy as np
import matplotlib.pyplot as plt

#Graphical parameters
mrkSz=5
pltSlcFactor=500
cm = 1/2.54
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15*cm, 15*cm), constrained_layout=True)
fontSize = 10
plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=1
numCreepPoints=50 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=1 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=5
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints))
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]

#Initialize compliance vector
complianceSeries = []
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
#Visualize data being stored, if you wish
for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Ecm_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Ecm for t in timeSpan]
    initialStrain=[1/costConcrete.Ecm_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=np.array([value for value in timeSpan])
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])

    #np.random.seed(12)
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element+np.random.normal(0,0.01*element, 1)[0]) for element in yaxis]
    J[enum]=np.array(J[enum])
    
    #Visualize data being stored, if you wish
    
    c = next(color)
    #plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    ax1.plot(xaxis[::pltSlcFactor]-age,J[enum][::pltSlcFactor]*1e6,label="EC2-$t_{0}$="+"{:.1f}".format(t0)+"d", marker='o',markersize=mrkSz,c=c)
    
ax1.set_xlabel("Days since loading - [t-$t_{0}$] (days)")
ax1.set_ylabel("Compliance - [J] ($10^{-6}$ $\cdot$ $MPa^{-1}$)")
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
         fancybox=True, columnspacing=0.5, ncol=1, fontsize=fontSize)
ax1.grid(which='minor', alpha=0.2)
ax1.grid(which='major', alpha=0.5)
fig.tight_layout()

#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel = fkc.kelvinChainModel(ageingComplianceSeries)

pltSlcFactor=1000

#Fit a classical DPL 
kelvinModel.fitDoublePowerLaw(graphVisualization=True, typeOfDPL='modified', mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel.fitDirichletSeries(kelvinModel.modifiedDoublePowerLawModel, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], creepTimesInterval=[1e-5, 365*numYearsOfCreep, numYearsOfCreep*(365*numCreepPoints)], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)
#kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)

ax2.set_xlabel("Days since loading - [t-$t_{0}$] (days)")
#ax2.set_ylabel("Compliance - [J] ($10^{-6}$ $\cdot$ $MPa^{-1}$)")
#ax2.set_ylabel("Compliance ($MPa^{-1}$)")
ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, columnspacing=0.2, ncol=2, fontsize=fontSize)
ax2.grid(which='minor', alpha=0.2)
ax2.grid(which='major', alpha=0.5)
fig.subplots_adjust(wspace=0.0, hspace=0, right=.95)

fig.tight_layout()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\renan\AppData\Local\Temp\ipykernel_12340\3583635213.py:79: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


In [29]:
print("Sum of Dirichlet series modulus:")
[print(sum(series)) for series in kelvinModel.dirichletSeriesModel['agingModulus']]
print("Dirichlet series modulus:")
print(kelvinModel.dirichletSeriesModel['agingModulus'])
print("Dirichlet series retardation times:")
print(kelvinModel.dirichletSeriesModel['retardationTimes'])
print("DPL E0 value:")
print(kelvinModel.modifiedDoublePowerLawModel['E0']/kelvinModel.modifiedDoublePowerLawModel['conversionFactor_compliance'])

Sum of Dirichlet series modulus:
1130.5843939775627
1240.4545662073883
1360.8822543630447
1492.9208307388951
1637.722316715404
1796.5446386605518
1970.7610507958664
2161.878846583118
Dirichlet series modulus:
[array([  1.67660447, 571.11833175, 275.55143399, 145.36056577,
        69.00246538,  37.46860242,  17.06050496,  10.18419629,
         3.16168894]), array([  1.94673695, 626.56545476, 302.30331687, 159.47286295,
        75.70156906,  41.10623429,  18.71682251,  11.17292767,
         3.46864114]), array([  2.13434897, 687.39506934, 331.65255476, 174.95521207,
        83.05105501,  45.09702744,  20.533943  ,  12.25765002,
         3.80539375]), array([  2.2588908 , 754.13101121, 363.85082458, 191.94079125,
        91.11404282,  49.47526946,  22.52747812,  13.44768256,
         4.17483994]), array([  2.3393331 , 827.34602227, 399.17526382, 210.57532942,
        99.95983898,  54.27857058,  24.71455554,  14.75324923,
         4.58015377]), array([  2.3904611 , 907.66918612, 437.928955

In [5]:
cm = 1/2.54
fontSize = 12
fig, (ax1) = plt.subplots(1, 1, sharey=True, figsize=(15*cm, 11*cm), constrained_layout=True)

plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
for enum, chain in  enumerate(kelvinModel.dirichletSeriesModel['agingModulus']): 
    c = next(color)
    #modulusSeries=[modulus[enum] for modulus in kelvinModel.dirichletSeriesModel['agingModulus']]
    plt.plot(kelvinModel.dirichletSeriesModel['retardationTimes'], np.array(chain)/kelvinModel.modifiedDoublePowerLawModel['conversionFactor_compliance'], marker="x", label="Loading age "+r'$t_{0}$='+"{:.1f}".format(kelvinModel.dirichletSeriesModel['t_line'][enum])+"d", c=c, linestyle="none")
    #/kelvinModel.dirichletSeriesModel['conversionFactor_compliance']
ax1.grid(which='minor', alpha=0.2)
ax1.grid(which='major', alpha=0.5)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.yscale('log')
plt.xscale('log')
plt.xlabel("Retardation time - ["+r'$\tau_{i}$'+"] (days)")
plt.ylabel("Kelvin unit modulus - ["+r'$E_{i}$'+"] (MPa)")
plt.show()


# 3. See retardation spectrum

## 3.1 Eurocode 2 compliance

### 3.1.1 Adjust a clean EC2 Kelvin chain

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import fitKelvinChain_v2025 as fkc
import creepEC2 as cEC2
import numpy as np
import matplotlib.pyplot as plt

mrkSz=5

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=100
numCreepPoints=5 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=3 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=15
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints))
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]

#Initialize compliance vector
complianceSeries = []
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
#Visualize data being stored, if you wish
for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Ecm_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Ecm for t in timeSpan]
    initialStrain=[1/costConcrete.Ecm_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=np.array([value for value in timeSpan])
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])

    #Visualize data being stored, if you wish
    c = next(color)
    #plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    plt.plot(xaxis[::pltSlcFactor]-age,J[enum][::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='o',markersize=mrkSz,c=c)
    plt.xlabel("Days since loading (days)")
    plt.ylabel("Compliance ($MPa^{-1}$)")
    

'''
plt.legend()
plt.plot()
'''

#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel_EC2clean = fkc.kelvinChainModel(ageingComplianceSeries)

pltSlcFactor=1000

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}

kelvinModel_EC2clean.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], retardationTimesFactor=0.5, graphVisualization= True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 3.1.2 Plot the retardation spectrum of EC2

In [23]:
retardationTimes=kelvinModel_EC2clean.dirichletSeriesModel['retardationTimes']
agingModulus=kelvinModel_EC2clean.dirichletSeriesModel['agingModulus']
loadingAges=kelvinModel_EC2clean.dirichletSeriesModel['t_line']

color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loadingAges))))
for enum, series in enumerate(agingModulus):
    c = next(color)
    plt.plot(retardationTimes, 1/np.array(series), label="{:.1f}".format(loadingAges[enum]), c=c)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

NameError: name 'kelvinModel_EC2clean' is not defined

## 3.2 MC2010 compliance

### 3.2.1 Adjust a clean MC2010 Kelvin Chain

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

#Graphical parameters
mrkSz=5
pltSlcFactor=1000

import fitKelvinChain_v2025 as fkc
import creepMC2010 as MC2010
import numpy as np
import matplotlib.pyplot as plt

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = MC2010.creepMC2010(250*250, 2*500, 'C50', '42.5N', 60, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=10
numCreepPoints=10 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=3 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=3
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints)) 
#Initialize compliance vector
complianceSeries=[]
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))

for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory)) for t in timeSpan]
    initialStrain=[1/costConcrete.Eci_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=[value for value in timeSpan]
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])

    #Visualize data being stored, if you wish
    c = next(color)
    plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="MC20-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    plt.xlabel("Days since loading (days)")
    plt.ylabel("Compliance ($MPa^{-1}$)")

#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel_MC2010clean = fkc.kelvinChainModel(ageingComplianceSeries)

#Fit a Dirichlet series
fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel_MC2010clean.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], retardationTimesFactor=0.5, graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor)

### 3.2.2 Plot the retardation spectrum of MC2010

In [ ]:
retardationTimes=kelvinModel_MC2010clean.dirichletSeriesModel['retardationTimes']
agingModulus=kelvinModel_MC2010clean.dirichletSeriesModel['agingModulus']
loadingAges=kelvinModel_MC2010clean.dirichletSeriesModel['t_line']

color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loadingAges))))
for enum, series in enumerate(agingModulus):
    c = next(color)
    plt.plot(retardationTimes, series, label="{:.1f}".format(loadingAges[enum]), c=c)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

# 4. Exploring poor definition of retardation times

## 4.1 Fit a Kelvin chain to a EC2 compliance

In [ ]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import fitKelvinChain_v2025 as fkc
import creepEC2 as cEC2
import numpy as np
import matplotlib.pyplot as plt

#Graphical parameters
mrkSz=5
pltSlcFactor=1
cm = 1/2.54
fig, ax1 = plt.subplots(1, 1, sharey=True, figsize=(10*cm, 11*cm), constrained_layout=True)
fontSize = 10
plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

#Create EC2 compliances
#This is a concrete with personalized E-Modulus evolution by a three parameter law
costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, modulusCalculatedPerCode=False, considerTemperatureEffect=False, α_ThreeParModulus=36000, β_ThreeParModulus=1, 𝜏_ThreeParModulus=28)

#This is a standard Eurocode 2 concrete
#costConcrete = cEC2.creepEC2(250*250, 2*500, 'C50', '42.5N', 100, aggregateType='limestone',considerTemperatureEffect=False)
timeHistory=[2000]
temperatureHistory=[20]

#Set time scale of the simulation
numYearsOfCreep=1
numCreepPoints=10 #This value is used in the logarithmic scale, so it is hard to explain what is it. The higher the more creep points the creep curves will have
initialLoadingAge=1 #days
finalLoadingAge=90 #days
numOfLoadingAgesTested=5
initialStrain=[]
loading_age = np.geomspace(initialLoadingAge, finalLoadingAge,num=numOfLoadingAgesTested)
#timeSpanBase = np.geomspace((1e-5),(365*numYearsOfCreep),num=numYearsOfCreep*(365*numCreepPoints))
timeSpanBeginning = np.arange((1e-5),1/(24*15),step=1/(24*60*60))
timeSpanLast = np.arange(1/(24*60*15),30,step=0.01)
timeSpanBase = np.concatenate((timeSpanBeginning,timeSpanLast), axis=None)
creepTimeSeries = [[] for age in loading_age]
J=[[] for ag in loading_age]

#Initialize compliance vector
complianceSeries = []
#Generate EC2 data to fit in DPL
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
#Visualize data being stored, if you wish
for enum, age in enumerate(loading_age):
    #Build the time span vector for this loading age
    timeSpan = timeSpanBase+age
    t0 = age #Just a dummy variable so everything looks beautiful inside the function call below

    fullCompliance = [(1/costConcrete.Ecm_computeEvolution(t0))+(costConcrete.ϕ_compute(t,t0,timeHistory,temperatureHistory))/costConcrete.Ecm for t in timeSpan]
    initialStrain=[1/costConcrete.Ecm_computeEvolution(t0)]

    #Organizes vectors to be stored. We manually include "elastic" deformation, imediately after load application
    xaxis=np.array([value for value in timeSpan])
    yaxis=fullCompliance
    complianceSeries.append([xaxis,yaxis])

    #np.random.seed(12)
    [creepTimeSeries[enum].append(element) for element in xaxis]
    creepTimeSeries[enum]=np.array(creepTimeSeries[enum])
    [J[enum].append(element) for element in yaxis]
    J[enum]=np.array(J[enum])
    
    #Visualize data being stored, if you wish
    
    c = next(color)
    #plt.plot(xaxis[::pltSlcFactor]-age,np.array(yaxis)[::pltSlcFactor]*1e6,label="EC2-"+"{:.1f}".format(t0)+"d", marker='x',markersize=mrkSz,c=c)
    ax1.plot(xaxis[::pltSlcFactor]-age,J[enum][::pltSlcFactor]*1e6,label="EC2-$t_{0}$="+"{:.1f}".format(t0)+"d", marker='o',markersize=mrkSz,c=c)
    
#Build the ageingComplianceSeries to feed to kelvinChainModel object
ageingComplianceSeries = [[t_line, complianceSeriesData] for t_line, complianceSeriesData in zip(loading_age,complianceSeries)]

#Instantiate kelvinChainModel object
kelvinModel = fkc.kelvinChainModel(ageingComplianceSeries)

pltSlcFactor=1000

#Fit a classical DPL 
#kelvinModel.fitDoublePowerLaw(graphVisualization=True, typeOfDPL='modified', mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)

fittingData = {'model':'raw', 't_line': loading_age, 't': creepTimeSeries, 'J': J}
kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax1)
#kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested],retardationTimesFactor=10,retardationTimesRange=[1e-9,0.00001,180], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax1)
#kelvinModel.fitDirichletSeries(kelvinModel.modifiedDoublePowerLawModel, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], creepTimesInterval=[1e-5, 365*numYearsOfCreep, numYearsOfCreep*(365*numCreepPoints)], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)
#kelvinModel.fitDirichletSeries(fittingData, loadingAgesInterval=[initialLoadingAge,finalLoadingAge, numOfLoadingAgesTested], graphVisualization=True, mrkSz=mrkSz, plotSlicingFactor=pltSlcFactor, axisObjectToPlot=ax2)

ax1.set_xlabel("Days since loading - [t-$t_{0}$] (days)")
ax1.set_ylabel("Compliance - [J] ($10^{-6}$ $\cdot$ $MPa^{-1}$)")
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, columnspacing=0.2, ncol=2, fontsize=fontSize)
ax1.grid(which='minor', alpha=0.2)
ax1.grid(which='major', alpha=0.5)
fig.tight_layout()


fig.tight_layout()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\renan\AppData\Local\Temp\ipykernel_17628\1810446156.py:99: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


In [7]:
plt.plot(timeSpan,fullCompliance)

In [23]:
cm = 1/2.54
fontSize = 12
fig, (ax1) = plt.subplots(1, 1, sharey=True, figsize=(10*cm, 11*cm), constrained_layout=True)

plt.rc('font', size=fontSize)
plt.rc('axes', titlesize=fontSize)
plt.rc('legend',**{'fontsize':fontSize})

color = iter(plt.cm.rainbow(np.linspace(0, 1, len(loading_age))))
for enum, chain in  enumerate(kelvinModel.dirichletSeriesModel['agingModulus']): 
    c = next(color)
    #modulusSeries=[modulus[enum] for modulus in kelvinModel.dirichletSeriesModel['agingModulus']]
    plt.plot(kelvinModel.dirichletSeriesModel['retardationTimes'], np.array(chain), marker="x", label="Loading age "+r'$t_{0}$='+"{:.1f}".format(kelvinModel.dirichletSeriesModel['t_line'][enum])+"d", c=c, linestyle="none")
    #/kelvinModel.dirichletSeriesModel['conversionFactor_compliance']
ax1.grid(which='minor', alpha=0.2)
ax1.grid(which='major', alpha=0.5)
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, columnspacing=0.2, ncol=1, fontsize=fontSize)
plt.yscale('log')
plt.xscale('log')
plt.xlabel("Retardation time - ["+r'$\tau_{i}$'+"] (days)")
plt.ylabel("Kelvin unit modulus - ["+r'$E_{i}$'+"] (MPa)")
plt.show()


C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\eventloops.py:128: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  el.exec() if hasattr(el, 'exec') else el.exec_()
C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\eventloops.py:128: UserWarning: The figure layout has changed to tight
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [32]:
kelvinModel.modifiedDoublePowerLawModel['conversionFactor_compliance']

TypeError: 'NoneType' object is not subscriptable